In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from vidgear.gears import CamGear
import cvzone
from polym import PolylineManager      

In [ ]:
# Initialize the video stream
stream = CamGear(source='https://www.youtube.com/watch?v=_TusTf0iZQU', stream_mode=True, logging=True).start()

# Load COCO class names
with open("coco.txt", "r") as f:
    class_names = f.read().splitlines()

# Load the YOLOv8 model
model = YOLO("yolo11s.pt")

# Create a PolylineManager instance
polyline_manager = PolylineManager()

# Set up the OpenCV window
cv2.namedWindow('RGB')

# Mouse callback to get mouse movements
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  # Add point on left mouse click
        polyline_manager.add_point((x, y))

# Set the mouse callback function
cv2.setMouseCallback('RGB', RGB)

going_up = {}
going_down = {}
gnu=[]
gnd=[]
while True:
    # Read a frame from the video stream
    frame = stream.read()
    

    frame = cv2.resize(frame, (1020, 500))

    # Run YOLOv11 tracking on the frame
    results = model.track(frame, persist=True, classes=[2])

    # Check if there are any boxes in the results
    if results[0].boxes is not None and results[0].boxes.id is not None:
        # Get the boxes (x, y, w, h), class IDs, track IDs, and confidences
        boxes = results[0].boxes.xyxy.int().cpu().tolist()           # Bounding boxes
        class_ids = results[0].boxes.cls.int().cpu().tolist()        # Class IDs
        track_ids = results[0].boxes.id.int().cpu().tolist()         # Track IDs
        confidences = results[0].boxes.conf.cpu().tolist()           # Confidence score

        # Draw boxes and labels on the frame
        for box, class_id, track_id, conf in zip(boxes, class_ids, track_ids, confidences):
            c = class_names[class_id]
            x1, y1, x2, y2 = box
            
            # Calculate the center of the bounding box
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            
            
            if polyline_manager.point_polygon_test((cx, cy), 'area1'):
                going_up[track_id] = (cx, cy)
            if track_id in going_up:
               if polyline_manager.point_polygon_test((cx, cy), 'area2'): 
                  cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                  cvzone.putTextRect(frame, f'{track_id}', (x1, y2), 1, 1)
                  cvzone.putTextRect(frame, f'{c}', (x1, y1), 1, 1)
                  if gnu.count(track_id)==0:
                     gnu.append(track_id)
                     
            if polyline_manager.point_polygon_test((cx, cy), 'area2'):
                going_down[track_id] = (cx, cy)
            if track_id in going_down:
               if polyline_manager.point_polygon_test((cx, cy), 'area1'): 
                  cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 2)
                  cvzone.putTextRect(frame, f'{track_id}', (x1, y2), 1, 1)
                  cvzone.putTextRect(frame, f'{c}', (x1, y1), 1, 1)
                  if gnd.count(track_id)==0:
                     gnd.append(track_id)
                
    godown=len(gnd)       
    goup=len(gnu)
    cvzone.putTextRect(frame, f'GoDown: {godown}', (50, 60), 2, 2)
    cvzone.putTextRect(frame, f'GoUp: {goup}', (50, 160), 2, 2)

    # Draw polylines and points on the frame
    frame = polyline_manager.draw_polylines(frame)

    # Display the frame
    cv2.imshow("RGB", frame)

    # Handle key events for polyline management
    if not polyline_manager.handle_key_events():
        break

key = cv2.waitKey(1)  # Wait for a key event
if key == ord('q'):  # Press 'q' to quit
    break

# Release the video capture object and close the display window
stream.stop()
cv2.destroyAllWindows()

14:17:12 ::    Helper     ::   INFO   :: Running VidGear Version: 0.3.3
14:17:12 ::    Helper     ::  DEBUG   :: Selecting `best` resolution for streams.
14:17:12 ::    CamGear    ::   INFO   :: Verifying Streaming URL using yt-dlp backend. Please wait...
14:17:14 ::    CamGear    ::   INFO   :: [Backend] :: Streaming URL is fully supported. Available Streams are: [144p, 240p, 360p, 480p, 720p, 1080p, best, worst]
14:17:14 ::    CamGear    :: WARNING  :: Livestream URL detected. It is strongly recommended to use the GStreamer backend (`backend=cv2.CAP_GSTREAMER`) with these URLs.
14:17:14 ::    CamGear    ::  DEBUG   :: Using `best` resolution for streaming.
14:17:14 ::    CamGear    ::  DEBUG   :: YouTube source ID: `_TusTf0iZQU`, Title: `г. Усть-Кут Online, Иркутская область - Пешеходный переход Школа #10 2024-11-09 14:17`, Quality: `best`
14:17:14 ::    CamGear    ::  DEBUG   :: Enabling Threaded Queue Mode for the current video source!



0: 320x640 4 cars, 355.1ms
Speed: 5.0ms preprocess, 355.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 203.0ms
Speed: 2.0ms preprocess, 203.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 386.5ms
Speed: 3.0ms preprocess, 386.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 314.8ms
Speed: 2.0ms preprocess, 314.8ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 228.8ms
Speed: 2.0ms preprocess, 228.8ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 309.5ms
Speed: 2.0ms preprocess, 309.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 262.8ms
Speed: 3.0ms preprocess, 262.8ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 199.9ms
Speed: 2.0ms preprocess, 199.9ms inference, 1.0ms postprocess per image at shape (1, 3, 32

Enter a name for the polyline:  area1



0: 320x640 5 cars, 271.8ms
Speed: 2.0ms preprocess, 271.8ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 cars, 311.9ms
Speed: 2.0ms preprocess, 311.9ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 cars, 320.8ms
Speed: 2.0ms preprocess, 320.8ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 332.4ms
Speed: 3.0ms preprocess, 332.4ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 342.1ms
Speed: 2.0ms preprocess, 342.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 352.8ms
Speed: 2.0ms preprocess, 352.8ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 178.6ms
Speed: 2.0ms preprocess, 178.6ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 219.7ms
Speed: 3.0ms preprocess, 219.7ms inference, 1.0ms postprocess per image at shape (1, 3, 32

Enter a name for the polyline:  area2



0: 320x640 4 cars, 60.4ms
Speed: 2.0ms preprocess, 60.4ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 61.6ms
Speed: 1.0ms preprocess, 61.6ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 56.3ms
Speed: 1.0ms preprocess, 56.3ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 58.5ms
Speed: 1.0ms preprocess, 58.5ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 53.4ms
Speed: 2.0ms preprocess, 53.4ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 63.5ms
Speed: 1.0ms preprocess, 63.5ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 73.1ms
Speed: 1.0ms preprocess, 73.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 74.7ms
Speed: 1.6ms preprocess, 74.7ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x